### Análisis y control del reporte de mantenimiento

In [ ]:
# ================================
# IMPORTACIÓN DE LIBRERÍAS
# ================================
import pandas as pd

In [ ]:
# ================================
# FUNCIÓN DE LIMPIEZA Y NORMALIZACIÓN DE DATASETS
# ================================
def limpiar_df(tabla):
    tabla.columns = tabla.columns.str.strip()
    tabla.columns = tabla.columns.str.replace(r'\s+', '_', regex=True)

    cols_texto = tabla.select_dtypes(include='object').columns
    tabla[cols_texto] = tabla[cols_texto].apply(lambda c: c.str.strip())

    return tabla

In [ ]:
# ================================
# CARGA DE ARCHIVOS DE ÓRDENES DE COMPRA Y SERVICIO
# ================================
oc = limpiar_df(pd.read_excel(r"D:\Users\AX.COSTOS3\Desktop\ABRAHAM\MANTENIMIENTO\OC_2025_2026.xlsx"))
seg_osr = limpiar_df(pd.read_excel(r"D:\Users\AX.COSTOS3\Desktop\ABRAHAM\MANTENIMIENTO\SEG_OSR_2025_2026.xlsx"))

In [ ]:
# ================================
# DEFINICIÓN DE PERÍODO DE ANÁLISIS
# ================================
año = 2026

In [ ]:
# ================================
# CÁLCULO DE COSTOS POR CENTRO DE COSTOS 
# ================================

# Centro de costos: MANGENELE
oc_cc_MANGENELE = oc[
    (oc["IDCONSUMIDOR"] == "MANGENELE") &
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"])) &
    (oc["FECHA"].dt.year == año) &
    (oc["AREA"].str.contains("MEJORAS", case=False, na=False))
    ]["SUBTOTALMEX"].sum()

osr_cc_MANGENELE = seg_osr[
    (seg_osr["IDCONSUMIDOR"] == "MANGENELE") &
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año)
    ]["IMPORTE_MEX"].sum()
suma_MANGELE = oc_cc_MANGENELE + osr_cc_MANGENELE

# Centro de costos: MANGENFRIO
oc_cc_MANGENFRIO = oc[
    (oc["IDCONSUMIDOR"] == "MANGENFRIO") &
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"]))  &
    (oc["FECHA"].dt.year == año) 
    ]["SUBTOTALMEX"].sum()

osr_cc_MANGENFRIO = seg_osr[
    (seg_osr["IDCONSUMIDOR"] == "MANGENFRIO") &
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año)
    ]["IMPORTE_MEX"].sum()
suma_MANGENFRIO = oc_cc_MANGENFRIO + osr_cc_MANGENFRIO

# Centro de costos: MANGENINF
oc_cc_MANGENINF = oc[
    (oc["IDCONSUMIDOR"] == "MANGENINF") &
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"]))  &
    (oc["FECHA"].dt.year == año)
    ]["SUBTOTALMEX"].sum()
osr_cc_MANGENINF = seg_osr[
    (seg_osr["IDCONSUMIDOR"] == "MANGENINF") &
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año)
    ]["IMPORTE_MEX"].sum()
suma_MANGENINF = oc_cc_MANGENINF + osr_cc_MANGENINF

# Centro de costos: MANGENMEC
oc_cc_MANGENMEC = oc[
    (oc["IDCONSUMIDOR"] == "MANGENMEC") &
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"])) &
    (oc["FECHA"].dt.year == año) 
    ]["SUBTOTALMEX"].sum()
osr_cc_MANGENMEC = seg_osr[
    (seg_osr["IDCONSUMIDOR"] == "MANGENMEC") &
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año)
    ]["IMPORTE_MEX"].sum()
suma_MANGENMEC = oc_cc_MANGENMEC + osr_cc_MANGENMEC


In [ ]:
# ================================
# CONSOLIDADO GENERAL DE MANTENIMIENTO Y MEJORAS
# ================================

# Subtotal de órdenes de servicio
osr_subtotal = seg_osr[
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año) &
    (seg_osr["IDCONSUMIDOR"].isin(["MANCONG","MANGEN","MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"])) &
    (seg_osr["AREA"].str.contains("MANTENIMIENTO|MEJORAS", case=False, na=False))
    ]["IMPORTE_MEX"].sum()

# Subtotal de órdenes de compra
oc_subtotal = oc[
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"])) &
    (oc["FECHA"].dt.year == año) &
    (oc["GRUPO"] != "ACTIVOS") &
    (oc["AREA"].str.contains("MANTENIMIENTO|MEJORAS", case=False, na=False))&
    (oc["IDCONSUMIDOR"].isin(["MANCONG","MANGEN","MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"]))
    ]["SUBTOTALMEX"].sum()

In [ ]:
# ================================
# CANTIDAD TOTAL DE ÓRDENES EMITIDAS
# ================================
cantidad_ordenes_osr = seg_osr[
    (seg_osr["ESTADO"] == "Conformidad") &
    (seg_osr["FECHA"].dt.year == año) &
    (seg_osr["IDCONSUMIDOR"].isin(["MANCONG","MANGEN","MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"])) &
    (seg_osr["AREA"].str.contains("MANTENIMIENTO|MEJORAS", case=False, na=False))
    ]["DOCUMENTO"].count()

cantidad_ordenes_oc = oc[
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial"])) &
    (oc["FECHA"].dt.year == año) &
    (oc["GRUPO"] != "ACTIVOS") &
    (oc["AREA"].str.contains("MANTENIMIENTO|MEJORAS", case=False, na=False))&
    (oc["IDCONSUMIDOR"].isin(["MANCONG","MANGEN","MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"]))
    ]["DOCUMENTO"].count()

total_ordenes = cantidad_ordenes_osr + cantidad_ordenes_oc


In [ ]:
# ================================
# SALIDA DE RESULTADOS CONSOLIDADOS
# ================================
print("---------------------------------")
print("CENTRO DE COSTOS DE OC Y OS")
print("---------------------------------")
print("EL SUBTOTAL OC/MTTO ES: ",oc_subtotal.round(2))
print("EL SUBTOTAL OS/MTTO ES: ",osr_subtotal.round(2))
print("TOTAL EMITIDO MTTO:     ",(oc_subtotal + osr_subtotal ).round(2))
print("---------------------------------")
print("CENTRO DE COSTOS DE MEJORAS")
print("---------------------------------")
print("SUMA DE MANGELE ES:     ",suma_MANGELE.round(2))
print("SUMA DE MANGENFRIO ES:  ",suma_MANGENFRIO.round(2))
print("SUMA DE MANGENINF ES:   ",suma_MANGENINF.round(2))
print("SUMA DE MANGENMEC ES:   ",suma_MANGENMEC.round(2))
print("---------------------------------")
print("TOTAL DE CENTRO MEJORAS:",(suma_MANGELE+suma_MANGENFRIO+suma_MANGENINF+suma_MANGENMEC).round(2))
print("TOTAL DE ORDENES OC_OSR:",total_ordenes + 7) # ESAS +7 SON ORDENES SON DEL 2024, YA ESTAN TRABAJANDO PARA ELIMINARLAS
print("---------------------------------")

---------------------------------
CENTRO DE COSTOS DE OC Y OS
---------------------------------
EL SUBTOTAL OC/MTTO ES:  4031.76
EL SUBTOTAL OS/MTTO ES:  2530.82
TOTAL EMITIDO MTTO:      6562.58
---------------------------------
CENTRO DE COSTOS DE MEJORAS
---------------------------------
SUMA DE MANGELE ES:      0.0
SUMA DE MANGENFRIO ES:   0.0
SUMA DE MANGENINF ES:    1066.6
SUMA DE MANGENMEC ES:    1156.16
---------------------------------
TOTAL DE CENTRO MEJORAS: 2222.76
TOTAL DE ORDENES OC_OSR: 70
---------------------------------


### Identificación y validación de órdenes de compra y servicio asociadas a mejoras

In [34]:
# DOCUMENTOS ORDENES DE SERVICIO
seg_osr_documentos = seg_osr[
    (seg_osr["ESTADO"].isin(["Conformidad","Aprobado"])) &
    (seg_osr["FECHA"] >= "2026-01-10") &
    (seg_osr["IDCONSUMIDOR"].isin(["MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"])) &
    (seg_osr["AREA"].str.contains("MANTENIMIENTO", case=False, na=False))
    ][["FECHA","DOCUMENTO","AREA","IDCONSUMIDOR","IMPORTE_MEX"]].drop_duplicates()
#DOCUMENTOS ORDENES DE COMPRA
oc_documentos = oc[
    (oc["ESTADO"].isin(["Atendido Total","Atendido Parcial","Aprobado"])) &
    (oc["FECHA"] >= "2026-01-10") &
    (oc["AREA"].str.contains("MANTENIMIENTO", case=False, na=False))&
    (oc["IDCONSUMIDOR"].isin(["MANGENMEC","MANGENINF","MANGENFRIO","MANGENELE"]))
    ][["FECHA","DOCUMENTO","AREA","IDCONSUMIDOR","SUBTOTALMEX"]].drop_duplicates()

seg_osr_documentos.reset_index(drop=True)

,FECHA,DOCUMENTO,AREA,IDCONSUMIDOR,IMPORTE_MEX
0,2026-01-12 16:20:34,OSR 0001-8179976,MTP - MANTENIMIENTO PREVENTIVO Y CORRECTIVO,MANGENINF,156.02
1,2026-01-12 16:20:34,OSR 0001-8179976,MTP - MANTENIMIENTO PREVENTIVO Y CORRECTIVO,MANGENINF,111.44


In [35]:
oc_documentos.reset_index(drop=True)

,FECHA,DOCUMENTO,AREA,IDCONSUMIDOR,SUBTOTALMEX
0,2026-01-26 11:36:56,OCO 0001-0169939,MTP - MANTENIMIENTO PREVENTIVO Y CORRECTIVO,MANGENINF,74.43
